In [4]:
from gurobipy import *
import numpy as np
import pandas as pd
from pandas import read_excel
from pandas import ExcelFile
import matplotlib.pyplot as plt
from pandas import DataFrame, ExcelWriter
from openpyxl import load_workbook

ModuleNotFoundError: No module named 'gurobipy'

# Function Define

In [3]:
def getSheetNames(excelfile):
    return (ExcelFile(excelfile)).sheet_names
def readExcelRange(excelfile,sheetname="Sheet1",startrow=1,endrow=1,startcol=1,endcol=1):
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]
def writeExcelData(df,excelfile,sheetname,startrow,startcol):
    #df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, na_rep='No values',header=False, index=False)
    writer.save()
    writer.close()

# Data Import

In [4]:
excelfile = r"Competition.xlsx"

In [5]:
sheets = getSheetNames(excelfile)

In [7]:
# Demand
demand = pd.read_excel(excelfile,sheet_name=sheets[0])
# Plant Capacity
p_cap = pd.read_excel(excelfile,sheet_name=sheets[1])
# Indound Cost
p_dc_cost = pd.read_excel(excelfile,sheet_name=sheets[2],skiprows=4)
p_dc_cost.set_index(["DC Candidates"], inplace=True)
# DC Handling Cost
han_cost = pd.read_excel(excelfile,sheet_name=sheets[3])
han_cost.set_index(["DC"], inplace=True)
# Outbound Cost
dc_c_cost = pd.read_excel(excelfile,sheet_name=sheets[4],skiprows=1,index_col=0)
# Delivery Time
time = pd.read_excel(excelfile,sheet_name=sheets[5],skiprows=1,index_col=0)

In [8]:
# Next Day Air Cost
next_day = pd.read_excel(excelfile,sheet_name=sheets[6],skiprows=1,index_col=0)

In [9]:
next_day.head()

,Atlanta,Charleston,Charlotte,Chattanooga,Chicago,Cincinnati,Dallas,Denver,Greenville,Indianapolis,Knoxville,Louisville,Memphis,Nashville,Pittsburgh
Customer,,,,,,,,,,,,,,,
10,2.58,2.38,2.58,2.58,2.58,2.58,2.82,2.82,2.58,2.58,2.58,2.58,2.75,2.58,2.38
12,1.90,1.78,1.90,1.90,1.90,1.90,2.06,2.06,1.90,1.90,1.90,1.90,2.03,1.90,1.78
14,1.95,1.81,1.95,1.95,1.95,1.95,2.13,2.13,1.95,1.95,1.95,1.95,2.06,1.95,1.81
18,1.68,1.57,1.68,1.68,1.68,1.68,1.86,1.86,1.68,1.68,1.68,1.68,1.82,1.68,1.57
19,3.81,3.81,3.81,3.81,3.81,3.81,4.26,4.26,3.81,3.81,3.81,3.81,4.08,3.81,3.57


# Optimization with Gurobi

In [19]:
results = {}

## 1. Optimization without time constraint or DC number constraint

### Get Data

In [16]:
plants = [p_cap.iloc[0,0],p_cap.iloc[1,0]]

In [17]:
# Inbound Routes
pdc = {}
for p in plants:
    for d in list(p_dc_cost.index):
        pdc[p,d] = p_dc_cost.loc[d,p]+han_cost.loc[d,'Handling Cost']

In [18]:
# Outbound Routes
dcc = {}
for d in dc_c_cost.columns.tolist():
    for c in list(dc_c_cost.index):
        dcc[d,c] = dc_c_cost.loc[c,d]

### Model Define
Minimize Cost

In [20]:
m = Model('csr')
m.ModelSense = GRB.MINIMIZE
m.setParam('TimeLimit',7200)

Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 7200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100


#### Decision Variable Define

In [21]:
p_dc_volume = {}
for p,d in pdc.keys():
    p_dc_volume[p,d] = m.addVar(vtype=GRB.CONTINUOUS, name="x_{0}_{1}".format(p,d))

In [22]:
dc_c_volume = {}
for d,c in dcc.keys():
    dc_c_volume[d,c] = m.addVar(vtype=GRB.CONTINUOUS, name="x_{0}_{1}".format(d,c))

In [23]:
m.update()

#### Adding Constraint 

In [24]:
# sum of dc is less than plants capacity
for p in plants:
    total = sum([p_dc_volume[i,j] for i,j in p_dc_volume.keys() if i==p])
    m.addConstr(total,GRB.LESS_EQUAL,p_cap.iloc[0,1],str(p)+'capacity')
# sum of customer zone is less than corresponding dc
for k in dc_c_cost.columns.tolist():
    total_c = sum([dc_c_volume[m,n] for m,n in dc_c_volume.keys() if m==k])
    m.addConstr(total_c,GRB.LESS_EQUAL,sum([p_dc_volume[a,b] for a,b in p_dc_volume.keys() if b==k]),str(k)+'volume')
# meet demand? question is whether we should consider the profits
for d,c in dcc.keys():
    supply_c = sum([dc_c_volume[i,j] for i,j in dc_c_volume.keys() if j==c])
    m.addConstr(supply_c,GRB.GREATER_EQUAL,demand.Demand[demand.Customer==c],str(c)+'zone demand')

#### Objective Function

In [26]:
cost_pdc = quicksum(pdc[p,d]*p_dc_volume[p,d] for p,d in pdc.keys())
cost_dcc = quicksum(dcc[d,c]*dc_c_volume[d,c] for d,c in dcc.keys())
m.setObjective(cost_pdc+cost_dcc,GRB.MINIMIZE)

In [27]:
m.update()

#### Optimization

In [28]:
m.optimize()

Optimize a model with 7592 rows, 7605 columns and 121260 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 9e+06]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 7070 rows and 0 columns
Presolve time: 0.03s
Presolved: 522 rows, 7605 columns, 15210 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.605e+03
 Factor NZ  : 8.610e+03 (roughly 3 MBytes of memory)
 Factor Ops : 1.501e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.04 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with primal simplex
Solved in 1427 iterations and 0.04 seconds
Optimal objective  1.155495793e+07


#### Get Results

In [29]:
# Get the optimized minimum cost
results['all5days']=m.objVal

In [36]:
# Get Inbound Routes
p_dc_opt = np.zeros((15,2))
p_dc_opt = pd.DataFrame(p_dc_opt,index=list(p_dc_cost.index),columns=p_dc_cost.columns.tolist())
for d in list(p_dc_opt.index):
    for p in p_dc_opt.columns.tolist():
        p_dc_opt.loc[d,p] = p_dc_volume[p,d].x

In [33]:
# Get Outbound Routes
dc_c_opt = np.zeros((505,15))
dc_c_opt = pd.DataFrame(dc_c_opt,index=list(dc_c_cost.index),columns=dc_c_cost.columns.tolist())
for c in list(dc_c_opt.index):
    for d in dc_c_opt.columns.tolist():
        dc_c_opt.loc[c,d] = dc_c_volume[d,c].x

In [38]:
dc_c_opt

,Atlanta,Charleston,Charlotte,Chattanooga,Chicago,Cincinnati,Dallas,Denver,Greenville,Indianapolis,Knoxville,Louisville,Memphis,Nashville,Pittsburgh
10,0.0,33081.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,18595.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,12046.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,65862.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22170.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139581.0,0.0,0.0,0.0,0.0,0.0
22,0.0,25917.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,17900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,10691.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,18003.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [221]:
# Write Excel
writeExcelData(dc_c_opt, excelfile, sheetname="PtoDC", startrow=3,startcol=2)
writeExcelData(p_dc_opt, excelfile, sheetname="DCtoC", startrow=3,startcol=2)

## 2. Optimization with current two DCs

### Get Data

In [40]:
# Inbound Routes
pdc_ori = {}
for p in plants:
    for d in ['Denver','Pittsburgh']:
        pdc_ori[p,d] = p_dc_cost.loc[d,p]+han_cost.loc[d,'Handling Cost']
# Outbound Routes
dcc_ori = {}
for d in ['Denver','Pittsburgh']:
    for c in list(dc_c_cost.index):
        dcc_ori[d,c] = dc_c_cost.loc[c,d]

### Model Define

In [41]:
m_ori = Model('csr')
m_ori.ModelSense = GRB.MINIMIZE
m_ori.setParam('TimeLimit',7200)

# Define Decision Variables
p_dc_volume_ori = {}
for p,d in pdc_ori.keys():
    p_dc_volume_ori[p,d] = m_ori.addVar(vtype=GRB.CONTINUOUS, name="x_{0}_{1}".format(p,d))

dc_c_volume_ori = {}
for d,c in dcc_ori.keys():
    dc_c_volume_ori[d,c] = m_ori.addVar(vtype=GRB.CONTINUOUS, name="x_{0}_{1}".format(d,c))

m_ori.update()

# Add Constraints
# sum of dc is less than plants capacity
for p in plants:
    total = sum([p_dc_volume_ori[i,j] for i,j in p_dc_volume_ori.keys() if i==p])
    m_ori.addConstr(total,GRB.LESS_EQUAL,p_cap.iloc[0,1],str(p)+'capacity')
# sum of customer zone is less than corresponding dc
for k in dc_c_cost.columns.tolist():
    total_c = sum([dc_c_volume_ori[m,n] for m,n in dc_c_volume_ori.keys() if m==k])
    m_ori.addConstr(total_c,GRB.LESS_EQUAL,sum([p_dc_volume_ori[a,b] for a,b in p_dc_volume_ori.keys() if b==k]),str(k)+'volume')
# meet demand? question is whether we should consider the profits
for d,c in dcc_ori.keys():
    supply_c = sum([dc_c_volume_ori[i,j] for i,j in dc_c_volume_ori.keys() if j==c])
    m_ori.addConstr(supply_c,GRB.GREATER_EQUAL,demand.Demand[demand.Customer==c],str(c)+'zone demand')

# Define Objective Function
cost_pdc_ori = quicksum(pdc_ori[p,d]*p_dc_volume_ori[p,d] for p,d in pdc_ori.keys())
cost_dcc_ori = quicksum(dcc_ori[d,c]*dc_c_volume_ori[d,c] for d,c in dcc_ori.keys())
m_ori.setObjective(cost_pdc_ori+cost_dcc_ori,GRB.MINIMIZE)

m_ori.update()

# Optimization
m_ori.optimize()

Changed value of parameter TimeLimit to 7200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 1027 rows, 1014 columns and 3038 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 9e+06]
Presolve removed 518 rows and 0 columns
Presolve time: 0.00s
Presolved: 509 rows, 1014 columns, 2028 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.705469e+07   0.000000e+00      0s
     522    1.2569546e+07   0.000000e+00   0.000000e+00      0s

Solved in 522 iterations and 0.01 seconds
Optimal objective  1.256954632e+07


### Get Results

In [42]:
results['2DC5days']=m_ori.objVal

In [44]:
# Get Iutbound Routes
p_dc_ori = np.zeros((2,2))
p_dc_ori = pd.DataFrame(p_dc_ori,index=['Denver', 'Pittsburgh'],columns=p_dc_cost.columns.tolist())

for d in list(p_dc_ori.index):
    for p in p_dc_ori.columns.tolist():
        p_dc_ori.loc[d,p] = p_dc_volume_ori[p,d].x

In [43]:
# Get Outbound Routes
dc_c_ori = np.zeros((505,2))
dc_c_ori = pd.DataFrame(dc_c_ori,index=list(dc_c_cost.index),columns=['Denver','Pittsburgh'])

for c in list(dc_c_ori.index):
    for d in dc_c_ori.columns.tolist():
        dc_c_ori.loc[c,d] = dc_c_volume_ori[d,c].x

In [45]:
dc_c_ori

,Denver,Pittsburgh
10,0.0,33081.0
12,0.0,18595.0
14,0.0,12046.0
18,0.0,65862.0
19,0.0,22170.0
21,0.0,139581.0
22,0.0,25917.0
24,0.0,17900.0
27,0.0,10691.0
28,0.0,18003.0


In [266]:
# Write Excel
writeExcelData(dc_c_ori, excelfile, sheetname="DCtoC_ori", startrow=3,startcol=2)
writeExcelData(p_dc_ori, excelfile, sheetname="PtoDC_ori", startrow=3,startcol=2)

## 3. Optimization with time constraint

### Minimum Unit Cost with Time Constraint

In [335]:
def flex_cost(time,range1,range2):
    # replace time bigger than required with 1, then replace the cost with 'next_day'
    time_new = time.copy()
    for i in range(range2):
        if i <= range1:
            time_new = time_new.replace(i,0)
        else:
            time_new = time_new.replace(i,1)
        cost1 = next_day*time_new
    # replace time less than required with 1, then replace the cost with 'dc_c_cost'
    time_newer = time.copy()
    for i in range(range2):
        if i <= range1:
            time_newer = time_newer.replace(i,1)
        else:
            time_newer = time_newer.replace(i,0)
        cost2 = dc_c_cost * time_newer
    return cost1+cost2

#### 1 Day Shipping 

In [237]:
dcc_cost_1day = flex_cost(time,1,6)
dcc_cost_1day

,Atlanta,Charleston,Charlotte,Chattanooga,Chicago,Cincinnati,Dallas,Denver,Greenville,Indianapolis,Knoxville,Louisville,Memphis,Nashville,Pittsburgh
Customer,,,,,,,,,,,,,,,
10,2.58,2.38,2.58,2.58,2.58,2.58,2.82,2.82,2.58,2.58,2.58,2.58,2.75,2.58,2.38
12,1.90,1.78,1.90,1.90,1.90,1.90,2.06,2.06,1.90,1.90,1.90,1.90,2.03,1.90,1.78
14,1.95,1.81,1.95,1.95,1.95,1.95,2.13,2.13,1.95,1.95,1.95,1.95,2.06,1.95,1.81
18,1.68,1.57,1.68,1.68,1.68,1.68,1.86,1.86,1.68,1.68,1.68,1.68,1.82,1.68,1.57
19,3.81,3.81,3.81,3.81,3.81,3.81,4.26,4.26,3.81,3.81,3.81,3.81,4.08,3.81,3.57
21,1.98,1.82,1.98,1.98,1.98,1.98,2.18,2.18,1.98,1.98,1.98,1.98,2.11,1.98,1.82
22,1.66,1.57,1.66,1.66,1.66,1.66,1.81,1.81,1.66,1.66,1.66,1.66,1.78,1.66,1.57
24,1.87,1.74,1.87,1.87,1.87,1.87,2.03,2.03,1.87,1.87,1.87,1.87,2.00,1.87,1.74
27,2.46,2.29,2.46,2.46,2.46,2.46,2.72,2.72,2.46,2.46,2.46,2.46,2.66,2.46,2.29


#### 2 Days Shipping

In [111]:
dcc_cost_2day = flex_cost(time,2,6)
dcc_cost_2day

,Atlanta,Charleston,Charlotte,Chattanooga,Chicago,Cincinnati,Dallas,Denver,Greenville,Indianapolis,Knoxville,Louisville,Memphis,Nashville,Pittsburgh
Customer,,,,,,,,,,,,,,,
10,2.58,2.38,0.64,2.58,2.58,2.58,2.82,2.82,0.64,2.58,2.58,2.58,2.75,2.58,0.62
12,1.90,1.78,1.90,1.90,1.90,1.90,2.06,2.06,1.90,1.90,1.90,1.90,2.03,1.90,0.50
14,1.95,0.46,0.56,1.95,1.95,0.56,2.13,2.13,0.56,0.56,1.95,0.56,2.06,1.95,0.46
18,1.68,0.52,0.58,1.68,1.68,0.58,1.86,1.86,0.58,0.58,1.68,0.58,1.82,1.68,0.52
19,3.81,0.66,0.66,3.81,3.81,0.66,4.26,4.26,0.66,0.66,3.81,0.66,4.08,3.81,0.66
21,1.98,0.66,0.66,1.98,1.98,0.66,2.18,2.18,0.66,0.66,1.98,0.66,2.11,1.98,0.66
22,1.66,0.62,0.64,1.66,1.66,0.64,1.81,1.81,0.64,0.64,1.66,0.64,1.78,1.66,0.62
24,1.87,0.58,0.62,1.87,1.87,0.62,2.03,2.03,0.62,0.62,1.87,0.62,2.00,1.87,0.58
27,2.46,2.29,0.50,2.46,2.46,2.46,2.72,2.72,0.50,2.46,2.46,2.46,2.66,2.46,0.42


#### 3 Days Shipping

In [239]:
dcc_cost_3day = flex_cost(time,3,6)

#### 4 Days Shipping

In [240]:
dcc_cost_4day = flex_cost(time,4,6)

### Optimization

In [241]:
def lowest_cost(cost):
    dcc_new = {}
    for d in cost.columns.tolist():
        for c in list(cost.index):
            dcc_new[d,c] = cost.loc[c,d]
#cost_pdc = quicksum(pdc[p,d]*p_dc_volume[p,d] for p,d in pdc.keys())
    cost_dcc = quicksum(dcc_new[d,c]*dc_c_volume[d,c] for d,c in dcc_new.keys())
    m.setObjective(cost_pdc+cost_dcc,GRB.MINIMIZE)

In [112]:
def write_res_toExcel(suffix,write=True):
    
    opt_dcc = np.zeros((505,15))
    opt_dcc = pd.DataFrame(opt_dcc,index=list(dc_c_cost.index),columns=dc_c_cost.columns.tolist())

    for c in list(opt_dcc.index):
        for d in opt_dcc.columns.tolist():
            opt_dcc.loc[c,d] = dc_c_volume[d,c].x


    opt_pdc = np.zeros((15,2))
    opt_pdc = pd.DataFrame(opt_pdc,index=list(p_dc_cost.index),columns=p_dc_cost.columns.tolist())

    for d in list(opt_pdc.index):
        for p in opt_pdc.columns.tolist():
            opt_pdc.loc[d,p] = p_dc_volume[p,d].x
    
    if write:
        writeExcelData(opt_dcc, excelfile, sheetname="DCtoC_"+suffix, startrow=3,startcol=2)
        writeExcelData(opt_pdc, excelfile, sheetname="PtoDC_"+suffix, startrow=3,startcol=2)
    
    return opt_dcc,opt_pdc

#### 1 Day Shipping

In [3]:
lowest_cost(dcc_cost_1day)
m.update()
m.optimize()

NameError: name 'lowest_cost' is not defined

In [243]:
results['all1day']=m.objVal
opt_dcc_w1,opt_pdc_w1 = write_res_toExcel('w1',write=False)

#### 2 Days Shipping

In [244]:
lowest_cost(dcc_cost_2day)
m.update()
m.optimize()

Optimize a model with 7593 rows, 7620 columns and 121275 nonzeros
Variable types: 7605 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 9e+06]

Loaded MIP start with objective 1.4479e+07

Presolve removed 7071 rows and 15 columns
Presolve time: 0.24s
Presolved: 522 rows, 7605 columns, 15210 nonzeros
Variable types: 7605 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 731 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1.4479e+07 1.4479e+07  0.00%     -    0s

Explored 0 nodes (731 simplex iterations) in 0.38 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 1.4479e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.447902987000e+07, best 

In [245]:
results['all2days']=m.objVal
opt_dcc_w2,opt_pdc_w2 = write_res_toExcel('w2',write=False)

#### 3 Days Shipping

In [246]:
lowest_cost(dcc_cost_3day)
m.update()
m.optimize()

Optimize a model with 7593 rows, 7620 columns and 121275 nonzeros
Variable types: 7605 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 9e+06]

Loaded MIP start with objective 1.15791e+07

Presolve removed 7071 rows and 15 columns
Presolve time: 0.33s
Presolved: 522 rows, 7605 columns, 15210 nonzeros
Variable types: 7605 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 807 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1.1579e+07 1.1579e+07  0.00%     -    0s

Explored 0 nodes (807 simplex iterations) in 0.48 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 1.15791e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.157910402000e+07, bes

In [247]:
results['all3day']=m.objVal
opt_dcc_w3,opt_pdc_w3 = write_res_toExcel('w3',write=False)

#### 4 Days Shipping

In [248]:
lowest_cost(dcc_cost_4day)
m.update()
m.optimize()

Optimize a model with 7593 rows, 7620 columns and 121275 nonzeros
Variable types: 7605 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 9e+06]

Loaded MIP start with objective 1.1555e+07

Presolve removed 7071 rows and 15 columns
Presolve time: 0.29s
Presolved: 522 rows, 7605 columns, 15210 nonzeros
Variable types: 7605 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 767 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1.1555e+07 1.1555e+07  0.00%     -    0s

Explored 0 nodes (767 simplex iterations) in 0.44 seconds
Thread count was 12 (of 12 available processors)

Solution count 1: 1.1555e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.155495793000e+07, best 

In [249]:
results['all4day']=m.objVal
opt_dcc_w4,opt_pdc_w4 = write_res_toExcel('w4',write=False)